In [1]:
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
import string
import math
import matplotlib.pyplot as plt
import re
import scipy
import math
import requests
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.manifold import MDS
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Model
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

2024-07-17 17:47:56.643698: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-17 17:47:56.643826: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-17 17:47:56.793715: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
!pip install transformers

In [4]:
from transformers import pipeline
generator = pipeline('text-generation',model='gpt2')

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
doc1 = generator('Olympics', max_length=10, num_return_sequences=1,truncation=True)
doc1 = doc1[0]['generated_text']
doc2 = generator('Corona Virus', max_length=10, num_return_sequences=1,truncation=True)
doc2 = doc2[0]['generated_text']
doc3 = generator('Egypt Pyramids', max_length=10, num_return_sequences=1,truncation=True)
doc3 = doc3[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [6]:
class Corpus:
    def __init__(self, list_of_docs):
        self.list_of_docs = [doc.lower() for doc in list_of_docs]
        self.N= len(list_of_docs)
    def Get_corpus(self):
        all_docs=""
        for i in range(self.N):
            all_docs+=self.list_of_docs[i]+" "
        return all_docs



In [7]:
documents = [doc1, doc2,doc3]
corpus=Corpus(documents)

In [8]:
def prepocessing(text, stopWords = set(stopwords.words('english')),unique_words_option=True):
    text=text.lower()
    text=text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\w*\d\w*' , ' ' , text)
    text = re.sub(r'[0-9]' , ' ' , text)
    text = re.sub(r'[^a-zA-Z0-9\s]' , ' ' , text)
    text = re.sub(r'\s+' , ' ', text)
    text = word_tokenize(text)
    text = [i for i in text if i != '']
    if unique_words_option: unique_words= set(text)
    else:return text
    return unique_words


In [9]:
vocab=prepocessing(corpus.Get_corpus())

In [10]:
vocab=list(vocab)
vocab.sort()
vocab

['a',
 'always',
 'are',
 'as',
 'b',
 'corona',
 'egypt',
 'envoy',
 'important',
 'in',
 'its',
 'not',
 'of',
 'olympics',
 'pyramids',
 'stars',
 'team',
 'the',
 'to',
 'virus']

In [11]:
tfidf_vectorizer = TfidfVectorizer()

tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

print("TF-IDF matrix:")
print(tfidf_matrix.toarray())
print()

feature_names =tfidf_vectorizer.get_feature_names_out()
print(feature_names)

TF-IDF matrix:
[[0.37796447 0.37796447 0.37796447 0.         0.         0.
  0.         0.37796447 0.         0.37796447 0.         0.37796447
  0.         0.         0.37796447 0.         0.         0.        ]
 [0.         0.         0.         0.40824829 0.         0.40824829
  0.         0.         0.         0.         0.40824829 0.
  0.         0.40824829 0.         0.40824829 0.         0.40824829]
 [0.         0.         0.         0.         0.4472136  0.
  0.4472136  0.         0.4472136  0.         0.         0.
  0.4472136  0.         0.         0.         0.4472136  0.        ]]

['always' 'are' 'as' 'corona' 'egypt' 'envoy' 'important' 'in' 'it' 'not'
 'of' 'olympics' 'pyramids' 'stars' 'team' 'the' 'to' 'virus']


In [12]:
def Calculate_TF_IDF(doc):
    doc_words=prepocessing(doc,False)
    counter = Counter(doc_words)
    tf = [counter[word] for word in vocab]
    idf=[]
    for word in vocab:
        nt = sum(1 for doc in corpus.list_of_docs if re.search(r'\b{}\b'.format(re.escape(word)), doc))
        idf.append(math.log((1+corpus.N )/(1+nt))+1)
    idf=np.array(idf)
    tf_idf=tf*idf
    normalized_tf_idf=tf_idf/np.sqrt(np.sum(np.square(tf_idf)))
    return normalized_tf_idf





In [13]:
print("TF-IDF matrix:")
for i in range (corpus.N):
    print(Calculate_TF_IDF(corpus.list_of_docs[i]))


TF-IDF matrix:
[0.35355339 0.35355339 0.35355339 0.35355339 0.         0.
 0.         0.         0.         0.35355339 0.         0.35355339
 0.         0.35355339 0.         0.         0.35355339 0.
 0.         0.        ]
[0.         0.         0.         0.         0.37796447 0.37796447
 0.         0.37796447 0.         0.         0.         0.
 0.37796447 0.         0.         0.37796447 0.         0.37796447
 0.         0.37796447]
[0.         0.         0.         0.         0.         0.
 0.40871302 0.         0.40871302 0.         0.57603355 0.
 0.         0.         0.40871302 0.         0.         0.
 0.40871302 0.        ]
